# 0. Enter to drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Github/Hackathon-BBVA-2022
!ls -a

/content/drive/MyDrive/Github/Hackathon-BBVA-2022
CHANGELOG.rst  House_prediction.ipynb  README.md
.git	       LICENSE		       requirements.txt


## 0.1. Connect to Github

In [108]:
!git checkout enzo

M	House_prediction.ipynb
Already on 'enzo'
Your branch is up to date with 'origin/enzo'.


In [109]:
!git status

On branch enzo
Your branch is up to date with 'origin/enzo'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   House_prediction.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [110]:
!git add .gitignore
!git add CHANGELOG.rst
!git add Dataset
!git add House_prediction.ipynb
!git add README.md
!git add requirements.txt

In [111]:
!git config --global user.email "enzo.cisneros@pucp.edu.pe"

In [112]:
!git commit -m "0.0.2"

[enzo 95dc807] 0.0.2
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite House_prediction.ipynb (89%)


In [113]:
!git status

On branch enzo
Your branch is ahead of 'origin/enzo' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


In [114]:
!git push --set-upstream origin enzo

Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 1.12 KiB | 191.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/ECisneros20/Hackathon-BBVA-2022
   7586660..95dc807  enzo -> enzo
Branch 'enzo' set up to track remote branch 'enzo' from 'origin'.


In [115]:
!git pull

Already up to date.


# 1. Data preprocessing

## 1.1. Loading data

In [100]:
#!gzip -dk Dataset/pe_properties.csv.gz

In [45]:
import pandas as pd

df_dataset = pd.read_csv('Dataset/pe_properties.csv')

In [46]:
df_dataset.head()

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,1qOI7X9aqAcO7PzjX1B29Q==,Propiedad,2021-06-03,2021-08-27,2021-06-03,NaN,NaN,Perú,Lima,Lima,...,5.0,NaN,NaN,450000.0,USD,NaN,CASA DE PLAYA VENDO EN LAS LAGUNAS,<b>CASA DE PLAYA VENDO EN LAS LAGUNAS</b><br><...,Casa,Venta
1,J350rcSI7Fx+92Wb1ev7rw==,Propiedad,2021-06-03,2021-07-07,2021-06-03,NaN,NaN,Perú,Arequipa,Arequipa,...,6.0,NaN,NaN,1050000.0,USD,NaN,HERMOSA CASA DE ENSUEÑOS EN CERRO COLORADO,<b>HERMOSA CASA DE ENSUEÑOS EN CERRO COLORADO<...,Casa,Venta
2,P+3USxoY2w5HQwg1N8JCtg==,Propiedad,2021-06-03,9999-12-31,2021-06-03,-8.095431,-79.05310,Perú,La Libertad,Trujillo,...,4.0,NaN,NaN,179000.0,USD,NaN,[ VENTA ]: ESTRENA TU CASA EN URB. SOL Y LUZ C...,<b>[ VENTA ]: ESTRENA TU CASA EN URB. SOL Y LU...,Casa,Venta
3,I9llUYb61GVgLNDUJ6CT5A==,Propiedad,2021-06-03,2021-06-04,2021-06-03,-13.840915,-76.25083,Perú,Ica,Ica,...,NaN,1120.0,NaN,52000.0,USD,Mensual,Lote venta,Hábitat VIp vende exclusivo terreno 1120 m2 e...,Lote,Venta
4,d7Sw3mL1Vq0dmzoXNpJUNQ==,Propiedad,2021-06-03,2021-06-04,2021-06-03,-12.799091,-76.56912,Perú,Lima,Lima,...,NaN,500.0,NaN,70000.0,USD,Mensual,Lote venta,Hábitat Vip Vende lote 500 m2 \r\nUbicado a ta...,Lote,Venta
